In [1]:
import pandas as pd
import glob
import os

data_dir = "data"
proj_files = glob.glob(os.path.join(data_dir, "FantasyPros_Fantasy_Football_Projections_*.csv"))

frames = []
for f in proj_files:
    # Skip the FLX file to avoid duplication
    if "FLX" in os.path.basename(f):
        continue
    # Read each projections file and tag it with its position
    df = pd.read_csv(f)
    position = os.path.basename(f).split("_")[-1].split(".")[0]
    df['Position'] = position
    frames.append(df)

projections_df = pd.concat(frames, ignore_index=True)


In [2]:
projections_df.columns

Index(['Player', 'Team', 'REC', 'YDS', 'TDS', 'ATT', 'YDS.1', 'TDS.1', 'FL',
       'FPTS', 'Position', 'CMP', 'INTS', 'ATT.1', 'FG', 'FGA', 'XPT', 'SACK',
       'INT', 'FR', 'FF', 'TD', 'SAFETY', 'PA', 'YDS_AGN'],
      dtype='object')

In [3]:
adp_df = pd.read_csv(os.path.join(data_dir, "FantasyPros_2025_Overall_ADP_Rankings.csv"))
merged_df = projections_df.merge(adp_df, on="Player", how="left")


In [4]:
merged_df.head()

,Player,Team_x,REC,YDS,TDS,ATT,YDS.1,TDS.1,FL,FPTS,...,Rank,Team_y,Bye,POS,ESPN,Sleeper,NFL,RTSports,FFC,AVG
0,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Ja'Marr Chase,CIN,6.7,86.4,0.7,0.2,1.2,0.0,0.1,12.8,...,1.0,CIN,10.0,WR1,1.0,1.0,1.0,1.0,NaN,1.0
2,Justin Jefferson,MIN,6.5,87.0,0.5,0.0,0.4,0.0,0.1,11.9,...,5.0,MIN,6.0,WR2,4.0,4.0,4.0,6.0,NaN,4.6
3,CeeDee Lamb,DAL,6.5,77.9,0.4,0.5,3.6,0.0,0.1,10.9,...,6.0,DAL,10.0,WR3,6.0,7.0,3.0,5.0,NaN,5.4
4,Puka Nacua,LAR,6.1,78.5,0.4,0.8,4.4,0.1,0.1,10.8,...,9.0,LAR,8.0,WR5,9.0,8.0,7.0,10.0,NaN,9.4


In [5]:
adp_df.columns

Index(['Rank', 'Player', 'Team', 'Bye', 'POS', 'ESPN', 'Sleeper', 'NFL',
       'RTSports', 'FFC', 'AVG'],
      dtype='object')

In [6]:
# Drop rows with missing player names
clean_proj_df = projections_df.dropna(subset=['Player']).copy()

# Strip whitespace from Player names
projections_df['Player'] = projections_df['Player'].astype(str).str.strip()

# Filter out rows where Player is empty or NaN
clean_proj_df = projections_df[projections_df['Player'].notna() & (projections_df['Player'] != '')].copy()

# Keep only columns with at least one non-NaN value
clean_proj_df = clean_proj_df.loc[:, clean_proj_df.notna().any()]

# Drop rows where Player is NaN
clean_adp_df = adp_df.dropna(subset=['Player']).copy()

# Use the AVG column as the ADP value
clean_adp_df = clean_adp_df.rename(columns={'AVG': 'ADP', 'POS': 'PosCategory'})

# Convert ADP values to numeric (any non-numeric or missing values become NaN)
clean_adp_df['ADP'] = pd.to_numeric(clean_adp_df['ADP'], errors='coerce')

# Extract the base position from values like 'WR1' or 'RB3'
clean_adp_df['Position'] = clean_adp_df['PosCategory'].str.extract(r'([A-Z]+)')

# Keep only the columns you need (player, position, ADP)
clean_adp_df = clean_adp_df[['Player', 'Position', 'ADP']]

In [7]:
merged_df = clean_proj_df.merge(clean_adp_df, on=['Player', 'Position'], how='left')

In [8]:
merged_df.head()

,Player,Team,REC,YDS,TDS,ATT,YDS.1,TDS.1,FL,FPTS,...,XPT,SACK,INT,FR,FF,TD,SAFETY,PA,YDS_AGN,ADP
0,Ja'Marr Chase,CIN,6.7,86.4,0.7,0.2,1.2,0.0,0.1,12.8,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
1,Justin Jefferson,MIN,6.5,87.0,0.5,0.0,0.4,0.0,0.1,11.9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.6
2,CeeDee Lamb,DAL,6.5,77.9,0.4,0.5,3.6,0.0,0.1,10.9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.4
3,Puka Nacua,LAR,6.1,78.5,0.4,0.8,4.4,0.1,0.1,10.8,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.4
4,Malik Nabers,NYG,6.1,79.1,0.4,0.2,1.6,0.0,0.1,10.7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.6


In [9]:
# After cleaning and merging...
# Select only the needed columns
columns_to_keep = ['Player', 'Team', 'Position', 'FPTS', 'ADP']
final_df = merged_df[columns_to_keep].copy()

# Drop rows where Player, Position, or FPTS is missing
final_df = final_df.dropna(subset=['Player', 'Position', 'FPTS'])

# You can either drop rows with missing ADP or assign them a high ADP to indicate “undrafted”
# For example:
final_df['ADP'] = final_df['ADP'].fillna(999)

# Optional: reset the index after dropping rows
final_df = final_df.reset_index(drop=True)

In [10]:
final_df.head()

,Player,Team,Position,FPTS,ADP
0,Ja'Marr Chase,CIN,WR,12.8,1.0
1,Justin Jefferson,MIN,WR,11.9,4.6
2,CeeDee Lamb,DAL,WR,10.9,5.4
3,Puka Nacua,LAR,WR,10.8,9.4
4,Malik Nabers,NYG,WR,10.7,9.6


In [11]:
from data_loader import load_clean_player_data

In [12]:
load_clean_player_data('data')

,Player,Team,Position,FPTS,ADP
0,Ja'Marr Chase,CIN,WR,12.8,1.0
1,Justin Jefferson,MIN,WR,11.9,4.6
2,CeeDee Lamb,DAL,WR,10.9,5.4
3,Puka Nacua,LAR,WR,10.8,9.4
4,Malik Nabers,NYG,WR,10.7,9.6
...,...,...,...,...,...
791,Baltimore Ravens,NaN,DST,5.2,135.8
792,New York Jets,NaN,DST,5.1,208.0
793,Carolina Panthers,NaN,DST,4.8,246.5
794,New Orleans Saints,NaN,DST,4.6,249.7


In [13]:
def generate_snake_order(n_teams: int, n_rounds: int) -> list[int]:
    """
    Generate a list of team indices representing the pick order for a snake draft.

    Parameters
    ----------
    n_teams : int
        Number of teams in the league.
    n_rounds : int
        Number of rounds in the draft (number of picks each team makes).

    Returns
    -------
    list[int]
        A list of length n_teams * n_rounds. Each element is the zero-based
        index of the team whose turn it is to pick at that slot.
    """
    order = []
    for round_num in range(n_rounds):
        # In even-numbered rounds (0-based), teams pick 0..n_teams-1
        # In odd-numbered rounds, teams pick in reverse order
        if round_num % 2 == 0:
            order.extend(range(n_teams))
        else:
            order.extend(reversed(range(n_teams)))
    return order


In [14]:
order = generate_snake_order(n_teams=10, )

TypeError: generate_snake_order() missing 1 required positional argument: 'n_rounds'

In [15]:
from data_loader import load_players_as_objects
from draft_state import DraftState
from draft_utils import calculate_rounds

# Load players
players = load_players_as_objects("data")

# Roster requirements and bench spots
starter_req = {"QB": 1, "RB": 2, "WR": 2, "TE": 1, "FLEX": 1, "DST": 1, "K": 1}
bench = 7

# Number of teams and your team’s index (0-based)
n_teams = 10
user_team_index = 3

# Initialize the draft state
draft_state = DraftState(players, starter_req, bench, n_teams, user_team_index)

# Check which team picks first and how many picks there are
print("Pick order (first 20 picks):", draft_state.draft_order[:20])
print("Total picks:", len(draft_state.draft_order))

# Make a pick for team 0 at pick 0 (for example)
first_player = draft_state.available_players[0]
if draft_state.make_pick(0, first_player):
    print(f"Team 0 drafted {first_player.name}")
    draft_state.advance_pick()

# You can query the current team on the clock:
print("Current team on clock:", draft_state.get_current_team_index())

Pick order (first 20 picks): [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 9, 8, 7, 6, 5, 4, 3, 2, 1, 0]
Total picks: 160
Team 0 drafted Ja'Marr Chase
Current team on clock: 1


## test functions

In [16]:
from data_loader import load_players_as_objects
players = load_players_as_objects("data")


In [17]:
from draft_state import DraftState

starter_requirements = {"QB":1, "RB":2, "WR":2, "TE":1, "FLEX":1, "DST":1, "K":1}
bench_spots = 7
n_teams = 10
user_team_index = 3  # adjust to where you draft

draft_state = DraftState(players, starter_requirements, bench_spots, n_teams, user_team_index)

In [18]:
from draft_simulator import recommend_players

recommendations = recommend_players(
    draft_state,
    starter_requirements=starter_requirements,
    candidate_pool_size=10,
    num_simulations=50,   # start small for testing
    weight_adp=-0.1,
    opponent_top_n=5
)
for player, exp_val in recommendations[:5]:
    print(f"{player.name:20s} {player.position}  Expected Value: {exp_val:.2f}")

Josh Allen           QB  Expected Value: 0.00
Lamar Jackson        QB  Expected Value: 0.00
Jayden Daniels       QB  Expected Value: 0.00
Jalen Hurts          QB  Expected Value: 0.00
Saquon Barkley       RB  Expected Value: 0.00


In [2]:
import random
random.seed(42)  # reproducible

from data_loader import load_players_as_objects
players = load_players_as_objects("data")

from draft_state import DraftState
starter_requirements = {"QB":1, "RB":2, "WR":2, "TE":1, "FLEX":1, "DST":1, "K":1}
bench_spots = 7
n_teams = 10
user_team_index = 3

draft_state = DraftState(players, starter_requirements, bench_spots, n_teams, user_team_index)

In [5]:
from draft_simulator import recommend_players

# Move the draft forward until it's your pick
user = draft_state.user_team_index
while draft_state.get_current_team_index() != user:
    # Advance without drafting to land exactly on your pick
    draft_state.advance_pick()

# Now run recommendations
recs = recommend_players(
    draft_state,
    starter_requirements=starter_requirements,
    candidate_pool_size=10,
    num_simulations=100,
    use_vor=True,
    weight_adp=-1.5,
    candidate_adp_margin=5,          # no need; you're on the clock
    candidate_pool_balanced=True,
    per_position_caps={"RB":4,"WR":4,"QB":2,"TE":2,"DST":1,"K":1},
    opponent_top_n=5,
    user_strategy_weight_adp=-0.5,
    user_strategy_top_n=5,
)
for p, ev in recs[:8]:
    print(f"{p.name:22s} {p.position:>3}  EV:{ev:.1f}  ADP:{p.adp:.1f}  FPTS:{p.fpts:.1f}")

Saquon Barkley          RB  EV:0.0  ADP:2.8  FPTS:17.0
Ja'Marr Chase           WR  EV:0.0  ADP:1.0  FPTS:12.8
Bijan Robinson          RB  EV:0.0  ADP:2.8  FPTS:15.2
Jahmyr Gibbs            RB  EV:0.0  ADP:4.6  FPTS:14.8
Justin Jefferson        WR  EV:0.0  ADP:4.6  FPTS:11.9
CeeDee Lamb             WR  EV:0.0  ADP:5.4  FPTS:10.9
Christian McCaffrey     RB  EV:0.0  ADP:9.4  FPTS:14.5
Puka Nacua              WR  EV:0.0  ADP:9.4  FPTS:10.8


In [12]:
from draft_simulator import evaluate_team_value

user = draft_state.user_team_index
# Find a candidate you can draft right now
cand = next((p for p in draft_state.available_players if draft_state.teams[user].can_draft(p)), None)
print("Candidate:", cand.name, cand.position if cand else None)

# Make the pick for your team
draft_state.make_pick(user, cand)
print("Removed from available?", cand not in draft_state.available_players)

# Inspect roster contents
print("Roster keys:", list(draft_state.teams[user].roster.keys()))
for pos, lst in draft_state.teams[user].roster.items():
    print(pos, len(lst), [p.name for p in lst[:3]])

# Evaluate value
val = evaluate_team_value(draft_state.teams[user], starter_requirements)
print("Team value after drafting", cand.name, ":", val)

Candidate: Ja'Marr Chase WR
Removed from available? True
Roster keys: ['QB', 'RB', 'WR', 'TE', 'FLEX', 'DST', 'K', 'BENCH']
QB 0 []
RB 0 []
WR 1 ["Ja'Marr Chase"]
TE 0 []
FLEX 0 []
DST 0 []
K 0 []
BENCH 0 []
Team value after drafting Ja'Marr Chase : 12.8


In [2]:
from draft_simulator import select_player_for_team

for _ in range(8):  # simulate 8 opponent picks
    t = draft_state.get_current_team_index()
    if t == user_team_index:
        break
    p = select_player_for_team(draft_state, t, top_n=5)
    if p:
        draft_state.make_pick(t, p)
    draft_state.advance_pick()

In [3]:
def advance_to_one_before_user_pick(ds):
    user = ds.user_team_index
    # Find your next pick index in the current state
    user_future_picks = [i for i in ds.get_pick_number_for_team(user) if i >= ds.current_pick_index]
    if not user_future_picks:
        return False  # no picks left
    next_user_pick = user_future_picks[0]
    # Advance the draft until we're exactly one pick before that
    while ds.current_pick_index < next_user_pick - 1:
        team_on_clock = ds.get_current_team_index()
        if team_on_clock == user:
            # Skip your early picks—don't draft, just advance to keep the test clean
            ds.advance_pick()
            continue
        p = select_player_for_team(ds, team_on_clock, top_n=5)
        if p:
            ds.make_pick(team_on_clock, p)
        ds.advance_pick()
    # Return True if we landed one pick before your turn
    return ds.current_pick_index == next_user_pick - 1

In [5]:
from draft_simulator import select_player_for_team

ok = advance_to_one_before_user_pick(draft_state)
print("At one pick before my turn?", ok, "Current idx:", draft_state.current_pick_index)
print("Team on clock now:", draft_state.get_current_team_index(), " (should NOT be me)")

At one pick before my turn? True Current idx: 2
Team on clock now: 2  (should NOT be me)


In [6]:
from draft_simulator import recommend_players

recs = recommend_players(
    draft_state,
    starter_requirements=starter_requirements,
    candidate_pool_size=10,
    num_simulations=100,      # small but you should see non-zero EVs now
    use_vor=True,
    weight_adp=-1.5,
    candidate_adp_margin=0,   # no need to filter; it's literally your turn next
    candidate_pool_balanced=True,
    per_position_caps={"RB":4,"WR":4,"QB":2,"TE":2,"DST":1,"K":1},
    opponent_top_n=5,
    user_strategy_weight_adp=-0.5,
    user_strategy_top_n=5,
)

for p, ev in recs[:8]:
    print(f"{p.name:22s} {p.position:>3}  EV:{ev:.1f}  ADP:{p.adp:.1f}  FPTS:{p.fpts:.1f}")


Bijan Robinson          RB  EV:0.0  ADP:2.8  FPTS:15.2
Jahmyr Gibbs            RB  EV:0.0  ADP:4.6  FPTS:14.8
Justin Jefferson        WR  EV:0.0  ADP:4.6  FPTS:11.9
CeeDee Lamb             WR  EV:0.0  ADP:5.4  FPTS:10.9
Christian McCaffrey     RB  EV:0.0  ADP:9.4  FPTS:14.5
Derrick Henry           RB  EV:0.0  ADP:10.4  FPTS:15.1
Puka Nacua              WR  EV:0.0  ADP:9.4  FPTS:10.8
Amon-Ra St. Brown       WR  EV:0.0  ADP:9.0  FPTS:10.2


In [3]:
from draft_simulator import recommend_players

recs = recommend_players(
    draft_state,
    starter_requirements=starter_requirements,
    candidate_pool_size=10,
    num_simulations=100,           # small while testing
    use_vor=True,                  # <-- turn on VOR
    weight_adp=-1.5,               # stronger ADP influence (tune later)
    candidate_adp_margin=8,        # ignore guys way before your pick
    candidate_pool_balanced=True,  # balanced per-position pool
    per_position_caps={"RB":4,"WR":4,"QB":2,"TE":2,"DST":1,"K":1},
    opponent_top_n=5,
    user_strategy_weight_adp=-0.5,
    user_strategy_top_n=5,
)

for p, ev in recs[:8]:
    print(f"{p.name:22s} {p.position:>3}  EV:{ev:.1f}  ADP:{p.adp:.1f}  FPTS:{p.fpts:.1f}")

Bijan Robinson          RB  EV:0.0  ADP:2.8  FPTS:15.2
Jahmyr Gibbs            RB  EV:0.0  ADP:4.6  FPTS:14.8
CeeDee Lamb             WR  EV:0.0  ADP:5.4  FPTS:10.9
Christian McCaffrey     RB  EV:0.0  ADP:9.4  FPTS:14.5
Derrick Henry           RB  EV:0.0  ADP:10.4  FPTS:15.1
Puka Nacua              WR  EV:0.0  ADP:9.4  FPTS:10.8
Amon-Ra St. Brown       WR  EV:0.0  ADP:9.0  FPTS:10.2
Malik Nabers            WR  EV:0.0  ADP:9.6  FPTS:10.7


In [6]:
from draft_simulator import copy_draft_state, evaluate_team_value

user = draft_state.user_team_index
candidate = next((p for p in draft_state.available_players
                  if draft_state.teams[user].can_draft(p)), None)
print("Candidate:", candidate.name, candidate.position)

sim = copy_draft_state(draft_state)

print("== check any equal in sim copy:",
      any(p == candidate for p in sim.available_players))
print("is check (identity):",
      any(p is candidate for p in sim.available_players))

# Use attribute-based match to grab the *sim copy* of the candidate:
def same(a,b):
    return (a.name, a.team, a.position) == (b.name, b.team, b.position)

cand_in_sim = next((p for p in sim.available_players if same(p, candidate)), None)
print("Found matching player object in sim?", cand_in_sim is not None)

Candidate: Ja'Marr Chase WR
== check any equal in sim copy: True
is check (identity): False
Found matching player object in sim? True


In [7]:
def force_candidate_once_and_eval(state, candidate):
    user = state.user_team_index
    s = copy_draft_state(state)

    # Move the copy to your pick (so no one snipes)
    while s.get_current_team_index() != user:
        s.advance_pick()

    print("On user pick? ->", s.get_current_team_index() == user)

    # In the copy, find the candidate object by attributes (avoid identity issues)
    def same(a,b): return (a.name,a.team,a.position)==(b.name,b.team,b.position)
    cand_sim = next((p for p in s.available_players if same(p, candidate)), None)
    print("Candidate present at pick?", cand_sim is not None)
    print("User can_draft(cand)?", s.teams[user].can_draft(candidate))

    if cand_sim:
        ok = s.make_pick(user, cand_sim)  # draft the *sim copy* object
        print("make_pick returned:", ok)
        s.advance_pick()
        val = evaluate_team_value(s.teams[user], starter_requirements)
        print("Value after forcing just candidate:", val)
        return ok, val
    return False, 0.0

ok, val = force_candidate_once_and_eval(draft_state, candidate)

On user pick? -> True
Candidate present at pick? True
User can_draft(cand)? True
make_pick returned: True
Value after forcing just candidate: 12.8


In [8]:
def debug_one_full_sim(state, candidate):
    import random
    user = state.user_team_index
    s = copy_draft_state(state)

    step = 0
    while not s.is_draft_over():
        team = s.get_current_team_index()
        step += 1
        if team == user:
            print(f"[STEP {step}] USER PICK")
            # Find candidate in sim’s available by attributes
            def same(a,b): return (a.name,a.team,a.position)==(b.name,b.team,b.position)
            cand_sim = next((p for p in s.available_players if same(p, candidate)), None)
            print("  candidate present?", cand_sim is not None)
            print("  can_draft?", s.teams[user].can_draft(candidate))
            if cand_sim and s.teams[user].can_draft(candidate):
                s.make_pick(user, cand_sim)
                print("  drafted candidate:", candidate.name)
            else:
                print("  FAILED to draft candidate, breaking")
                break
            s.advance_pick()
        else:
            # Opponent picks (just pick someone so we move forward)
            from draft_simulator import select_player_for_team
            p = select_player_for_team(s, team, top_n=5)
            print(f"[STEP {step}] OPP {team} picks:", p.name if p else None)
            if p: s.make_pick(team, p)
            s.advance_pick()

        # Quit after a handful of steps to keep logs small
        if step > 50: break

    val = evaluate_team_value(s.teams[user], starter_requirements)
    print("Final team value:", val)

# Try with your top candidate right now:
debug_one_full_sim(draft_state, candidate)


[STEP 1] USER PICK
  candidate present? True
  can_draft? True
  drafted candidate: Ja'Marr Chase
[STEP 2] OPP 4 picks: CeeDee Lamb
[STEP 3] OPP 5 picks: Saquon Barkley
[STEP 4] OPP 6 picks: Jahmyr Gibbs
[STEP 5] OPP 7 picks: Puka Nacua
[STEP 6] OPP 8 picks: Amon-Ra St. Brown
[STEP 7] OPP 9 picks: Christian McCaffrey
[STEP 8] OPP 9 picks: Ashton Jeanty
[STEP 9] OPP 8 picks: Malik Nabers
[STEP 10] OPP 7 picks: De'Von Achane
[STEP 11] OPP 6 picks: Brian Thomas Jr.
[STEP 12] OPP 5 picks: Bijan Robinson
[STEP 13] OPP 4 picks: Nico Collins
[STEP 14] USER PICK
  candidate present? False
  can_draft? True
  FAILED to draft candidate, breaking
Final team value: 12.8


In [9]:
import random
from draft_simulator import copy_draft_state, evaluate_team_value, score_players, select_player_for_team

def recommend_one_candidate_debug(ds, starter_requirements, candidate, *, num_sims=50,
                                  opponent_top_n=5, user_strategy_weight_adp=-0.5, user_strategy_top_n=5):
    user = ds.user_team_index
    successes = 0
    total_val = 0.0

    def same(a,b):  # attribute-based matcher to find the sim's copy
        return (a.name,a.team,a.position)==(b.name,b.team,b.position)

    for s_idx in range(num_sims):
        sim = copy_draft_state(ds)
        forced = False

        # Simulate to the end, forcing candidate at first user pick
        while not sim.is_draft_over():
            team = sim.get_current_team_index()
            if team == user:
                if not forced:  # force candidate now
                    cand_sim = next((p for p in sim.available_players if same(p, candidate)), None)
                    if cand_sim and sim.teams[user].can_draft(cand_sim):
                        sim.make_pick(user, cand_sim)
                        forced = True
                    else:
                        # candidate gone or ineligible this sim
                        sim.advance_pick()
                        break
                else:
                    # later user picks: simple user strategy
                    elig = [p for p in sim.available_players if sim.teams[user].can_draft(p)]
                    if elig:
                        scores = score_players(elig, weight_adp=user_strategy_weight_adp)
                        top = [p for p,_ in sorted(scores.items(), key=lambda x: x[1], reverse=True)[:user_strategy_top_n]]
                        sim.make_pick(user, random.choice(top))
                sim.advance_pick()
            else:
                opp = select_player_for_team(sim, team, top_n=opponent_top_n)
                if opp: sim.make_pick(team, opp)
                sim.advance_pick()

        if forced:
            val = evaluate_team_value(sim.teams[user], starter_requirements)
            total_val += val
            successes += 1

    print(f"successes: {successes}/{num_sims}, avg if drafted: {total_val/successes if successes else 0.0:.2f}")
    return successes, (total_val/successes if successes else 0.0)

# pick any player you can draft right now
user = draft_state.user_team_index
cand = next((p for p in draft_state.available_players if draft_state.teams[user].can_draft(p)), None)
recommend_one_candidate_debug(draft_state, starter_requirements, cand, num_sims=50)


successes: 50/50, avg if drafted: 84.49


(50, 84.49399999999997)

In [3]:
import random
random.seed(42)  # reproducible

from data_loader import load_players_as_objects
players = load_players_as_objects("data")

from draft_state import DraftState
starter_requirements = {"QB":1, "RB":2, "WR":2, "TE":1, "FLEX":1, "DST":1, "K":1}
bench_spots = 7
n_teams = 10
user_team_index = 9

draft_state = DraftState(players, starter_requirements, bench_spots, n_teams, user_team_index)

In [4]:
from draft_simulator import recommend_players

# while draft_state.get_current_team_index() != draft_state.user_team_index:
#     draft_state.advance_pick()

recs = recommend_players(
    draft_state,
    starter_requirements=starter_requirements,
    candidate_pool_size=12,
    num_simulations=50,
    use_vor=True,
    weight_adp=-1.2,
    candidate_adp_margin=4,
    candidate_pool_balanced=False,
    candidate_pool_dynamic=True,
    per_position_caps={"RB":4,"WR":4,"QB":2,"TE":2,"DST":1,"K":1},
    opponent_top_n=5,
    user_strategy_weight_adp=-0.5,
    user_strategy_top_n=5,
    allow_flex_early=True,
    flex_threshold=0.7,   # FLEX must beat best base by 0.7 VOR to be taken early
    allow_bench_early = False,
    bench_threshold = 1.0,
)

for r in recs:
    print(f"{r.player.name:22s} {r.player.position:>3}  "
          f"EV*:{r.ev_unconditional:.2f}  "
          f"P(avail):{r.p_available:.0%}  "
          f"VOR:{r.vor:.1f}  ADPΔ:{r.adp_delta:+.0f}  "
          f"{' | '.join(r.rationale[:2])}")


Josh Allen              QB  EV*:85.10  P(avail):100%  VOR:4.0  ADPΔ:-13  +4.0 VOR vs QB replacement | 100% chance available
Brock Bowers            TE  EV*:84.84  P(avail):100%  VOR:3.0  ADPΔ:-9  +3.0 VOR vs TE replacement | 100% chance available
Lamar Jackson           QB  EV*:84.76  P(avail):100%  VOR:3.8  ADPΔ:-12  +3.8 VOR vs QB replacement | 100% chance available
Trey McBride            TE  EV*:84.73  P(avail):100%  VOR:3.1  ADPΔ:-17  +3.1 VOR vs TE replacement | 100% chance available
Denver Broncos         DST  EV*:83.11  P(avail):100%  VOR:1.4  ADPΔ:-105  +1.4 VOR vs DST replacement | 100% chance available
Cameron Dicker           K  EV*:82.99  P(avail):100%  VOR:0.2  ADPΔ:-112  +0.2 VOR vs K replacement | 100% chance available
Brandon Aubrey           K  EV*:81.90  P(avail):100%  VOR:0.0  ADPΔ:-96  +0.0 VOR vs K replacement | 100% chance available
Philadelphia Eagles    DST  EV*:81.86  P(avail):100%  VOR:0.2  ADPΔ:-109  +0.2 VOR vs DST replacement | 100% chance available
Derric